# NASDAQ 100 Trend Classification: Machine Learning's Approach to Identifying Major Turning Points

In [1]:
import yfinance as yf
from fredapi import Fred
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt


## Fetching Data from Yahoo Finance

### Varible Description 

^NDX (NASDAQ 100 Index): This index consists of 100 of the largest domestic and international non-financial companies listed on the NASDAQ stock exchange based on market capitalization. It reflects companies across major industry groups, including technology, telecom, and biotechnology.

^DJT (Dow Jones Transportation Average): The DJTA is the oldest U.S. stock index, created by Charles Dow in 1884. It represents the stock performance of 20 large, U.S.-based transportation companies (railroads, shipping, logistics, airlines).

^VIX (CBOE Volatility Index): Often referred to as the "fear index", the VIX measures the stock market's expectation of 30-day forward-looking volatility. It's derived from the price inputs of the S&P 500 index options.

^RUT (Russell 2000 Index): The Russell 2000 index is a benchmark for small-cap stocks in the U.S. It measures the performance of the 2,000 smallest stocks in the Russell 3000 index.

^W5000 (Wilshire 5000 Total Market Index): This is a market-capitalization-weighted index that seeks to represent the total U.S. equity market. Often referred to as the "total market index", it represents the stocks of nearly every publicly traded company in the U.S.

^TNX (10-Year Treasury Note Yield): Represents the yield or interest rate on a U.S. 10-year treasury note. These yields influence various interest rates, including mortgages, and they reflect overall confidence in the U.S. economy.

^IRX (3-Month Treasury Bill Yield): Represents the yield or interest rate on a U.S. 3-month treasury bill. It's a short-term measure and is often used as a benchmark for short-term interest rates.

^FVX (2-Year Treasury Note Yield): Represents the yield or interest rate on a U.S. 2-year treasury note. Like other treasury yields, it's influenced by various economic factors and is often used to gauge short-term economic outlook.

GC=F (Gold Futures): Represents the gold futures contract price, which is a standardized contract to buy or sell gold at a specific price on a future date. Gold is often viewed as a hedge against inflation and currency fluctuations.


In [2]:
start_date = "2007-01-01"
end_date = "2023-08-10"

# Define the tickers
tickers = ["^NDX", "^DJT", "^VIX", "^RUT", "^W5000", "^TNX", "^IRX", "^FVX", "GC=F"] 

# Download the data
data = yf.download(tickers, start=start_date, end=end_date)

# Extract adjusted closing prices
adj_closing_data = data['Adj Close']

# Extract volume data
volume_data = data['Volume']


[*********************100%***********************]  9 of 9 completed


In [3]:
print(adj_closing_data.head())  
print(volume_data.head())     


                  GC=F         ^DJT   ^FVX   ^IRX         ^NDX        ^RUT  \
Date                                                                         
2007-01-02  635.200012          NaN    NaN    NaN          NaN         NaN   
2007-01-03  627.099976  4650.660156  4.657  4.915  1759.369995  787.419983   
2007-01-04  623.900024  4673.069824  4.605  4.900  1792.910034  789.950012   
2007-01-05  604.900024  4612.350098  4.644  4.910  1785.300049  775.869995   
2007-01-08  607.500000  4624.180176  4.658  4.910  1787.140015  776.989990   

             ^TNX   ^VIX        ^W5000  
Date                                    
2007-01-02    NaN    NaN           NaN  
2007-01-03  4.664  12.04  14246.709961  
2007-01-04  4.618  11.51  14269.900391  
2007-01-05  4.646  12.14  14164.799805  
2007-01-08  4.660  12.00  14197.150391  
              GC=F        ^DJT  ^FVX  ^IRX          ^NDX          ^RUT  ^TNX  \
Date                                                                           
2007-0

Some tickers don´t have volume data like traesury yields. We will proceed to clean this up before joinning the two DataFrames.

In [4]:
# Manually remove columns that have all NaN values
columns_to_drop = ['^FVX', '^IRX', '^TNX', '^W5000', '^VIX', 'GC=F']
volume_data_cleaned = volume_data.drop(columns=columns_to_drop, errors='ignore')

print(volume_data_cleaned.head())


                  ^DJT          ^NDX          ^RUT
Date                                              
2007-01-02         NaN           NaN           NaN
2007-01-03  40270000.0  2.435280e+09  3.429160e+09
2007-01-04  32590000.0  2.104210e+09  3.004460e+09
2007-01-05  27450000.0  2.060360e+09  2.919400e+09
2007-01-08  26060000.0  1.905620e+09  2.763340e+09


In [5]:
# Renaming the columns
volume_data_cleaned.rename(columns={
    '^DJT': 'Volume_^DJT',
    '^NDX': 'Volume_^NDX',
    '^RUT': 'Volume_^RUT'
}, inplace=True)

print(volume_data_cleaned.head())


            Volume_^DJT   Volume_^NDX   Volume_^RUT
Date                                               
2007-01-02          NaN           NaN           NaN
2007-01-03   40270000.0  2.435280e+09  3.429160e+09
2007-01-04   32590000.0  2.104210e+09  3.004460e+09
2007-01-05   27450000.0  2.060360e+09  2.919400e+09
2007-01-08   26060000.0  1.905620e+09  2.763340e+09


In [6]:
# Joining on the 'Date' index
merged_data_yahoo = adj_closing_data.merge(volume_data_cleaned, left_index=True, right_index=True)

print(merged_data_yahoo.head(42))


                  GC=F         ^DJT   ^FVX   ^IRX         ^NDX        ^RUT  \
Date                                                                         
2007-01-02  635.200012          NaN    NaN    NaN          NaN         NaN   
2007-01-03  627.099976  4650.660156  4.657  4.915  1759.369995  787.419983   
2007-01-04  623.900024  4673.069824  4.605  4.900  1792.910034  789.950012   
2007-01-05  604.900024  4612.350098  4.644  4.910  1785.300049  775.869995   
2007-01-08  607.500000  4624.180176  4.658  4.910  1787.140015  776.989990   
2007-01-09  613.099976  4632.660156  4.655  4.945  1795.630005  778.330017   
2007-01-10  611.599976  4641.470215  4.671  4.950  1816.150024  778.869995   
2007-01-11  612.400024  4693.040039  4.724  4.965  1834.859985  788.450012   
2007-01-12  625.500000  4760.270020  4.759  4.955  1844.810059  794.260010   
2007-01-16  624.599976  4861.839844  4.737  4.955  1842.439941  791.479980   
2007-01-17  632.099976  4813.620117  4.778  4.980  1827.680054  

# Fetching Data from NASDAQ

S&P PE (Price-to-Earnings) Ratio: 
The S&P PE Ratio represents the average price-to-earnings ratio of the companies included in the S&P 500 index. This ratio is calculated by taking the current price of the index and dividing it by its earnings per share (EPS). It's a key metric used by investors to evaluate the overall valuation of the stock market.

In [7]:
# Open the config file
with open('config.json', 'r') as file:
    config = json.load(file)

In [8]:
base_url = "https://data.nasdaq.com/api/v3/datasets"
nasdaq_api_key = config["NASDAQ_API_KEY"]  #extract key
start_date = "2007-01-01"
end_date = "2023-08-10"

endpoint = f"{base_url}/MULTPL/SP500_PE_RATIO_MONTH/data.json?start_date={start_date}&end_date={end_date}&api_key={nasdaq_api_key}"

response = requests.get(endpoint)

if response.status_code == 200:
    data = response.json()
    # For example, to extract the actual data values:
    values = data['dataset_data']['data']
    for value in values:
        date, pe_ratio = value[0], value[1]
        print(f"Date: {date}, PE Ratio: {pe_ratio}")
else:
    print("Failed to fetch data:", response.status_code, response.text)

Date: 2023-08-01, PE Ratio: 25.45
Date: 2023-07-31, PE Ratio: 26.56
Date: 2023-07-01, PE Ratio: 25.74
Date: 2023-06-30, PE Ratio: 25.76
Date: 2023-06-01, PE Ratio: 24.81
Date: 2023-05-31, PE Ratio: 24.2
Date: 2023-05-01, PE Ratio: 23.67
Date: 2023-04-01, PE Ratio: 23.53
Date: 2023-03-31, PE Ratio: 21.97
Date: 2023-03-01, PE Ratio: 22.66
Date: 2023-02-28, PE Ratio: 21.22
Date: 2023-02-01, PE Ratio: 23.4
Date: 2023-01-31, PE Ratio: 21.79
Date: 2023-01-01, PE Ratio: 22.82
Date: 2022-12-01, PE Ratio: 22.65
Date: 2022-11-30, PE Ratio: 21.22
Date: 2022-11-01, PE Ratio: 22.07
Date: 2022-10-31, PE Ratio: 20.14
Date: 2022-10-01, PE Ratio: 20.44
Date: 2022-09-30, PE Ratio: 18.12
Date: 2022-09-01, PE Ratio: 20.58
Date: 2022-08-31, PE Ratio: 19.98
Date: 2022-08-01, PE Ratio: 22.03
Date: 2022-07-01, PE Ratio: 20.53
Date: 2022-06-30, PE Ratio: 19.13
Date: 2022-06-01, PE Ratio: 20.28
Date: 2022-05-31, PE Ratio: 20.88
Date: 2022-05-01, PE Ratio: 20.81
Date: 2022-04-01, PE Ratio: 22.4
Date: 2022-03-31,

In [9]:
# Create a DataFrame for the S&P PE ratio
data_list = [{'Date': value[0], 'PE_Ratio': value[1]} for value in values]
pe_ratio_df = pd.DataFrame(data_list)

# Convert the date strings to datetime format
pe_ratio_df['Date'] = pd.to_datetime(pe_ratio_df['Date'])

# Set 'Date' as the index for pe_ratio_df
pe_ratio_df.set_index('Date', inplace=True)

print(pe_ratio_df.head(24))

            PE_Ratio
Date                
2023-08-01     25.45
2023-07-31     26.56
2023-07-01     25.74
2023-06-30     25.76
2023-06-01     24.81
2023-05-31     24.20
2023-05-01     23.67
2023-04-01     23.53
2023-03-31     21.97
2023-03-01     22.66
2023-02-28     21.22
2023-02-01     23.40
2023-01-31     21.79
2023-01-01     22.82
2022-12-01     22.65
2022-11-30     21.22
2022-11-01     22.07
2022-10-31     20.14
2022-10-01     20.44
2022-09-30     18.12
2022-09-01     20.58
2022-08-31     19.98
2022-08-01     22.03
2022-07-01     20.53


### Merging

In [10]:
# Merge the data
merged_data = merged_data_yahoo.merge(pe_ratio_df, left_index=True, right_index=True, how='left')

print(merged_data.head(42))

                  GC=F         ^DJT   ^FVX   ^IRX         ^NDX        ^RUT  \
Date                                                                         
2007-01-02  635.200012          NaN    NaN    NaN          NaN         NaN   
2007-01-03  627.099976  4650.660156  4.657  4.915  1759.369995  787.419983   
2007-01-04  623.900024  4673.069824  4.605  4.900  1792.910034  789.950012   
2007-01-05  604.900024  4612.350098  4.644  4.910  1785.300049  775.869995   
2007-01-08  607.500000  4624.180176  4.658  4.910  1787.140015  776.989990   
2007-01-09  613.099976  4632.660156  4.655  4.945  1795.630005  778.330017   
2007-01-10  611.599976  4641.470215  4.671  4.950  1816.150024  778.869995   
2007-01-11  612.400024  4693.040039  4.724  4.965  1834.859985  788.450012   
2007-01-12  625.500000  4760.270020  4.759  4.955  1844.810059  794.260010   
2007-01-16  624.599976  4861.839844  4.737  4.955  1842.439941  791.479980   
2007-01-17  632.099976  4813.620117  4.778  4.980  1827.680054  

The S&P PE ratio is provided monthly, while the other data is recorded daily. 
We will fill forward the missing values (NaN) for the PE_Ratio. 

First, we will fill in the NaN values for January with 17.36.

Then, use the fillna method with the method parameter set to ffill (which stands for forward fill).

In [11]:
# Step 1: Fill NaN values for January 2007 with 17.36
merged_data.loc['2007-01-01':'2007-01-31', 'PE_Ratio'] = merged_data.loc['2007-01-01':'2007-01-31', 'PE_Ratio'].fillna(17.36)

# Step 2: Forward fill the rest of the NaN values
merged_data['PE_Ratio'].fillna(method='ffill', inplace=True)


In [12]:
print(merged_data.head(13))

                  GC=F         ^DJT   ^FVX   ^IRX         ^NDX        ^RUT  \
Date                                                                         
2007-01-02  635.200012          NaN    NaN    NaN          NaN         NaN   
2007-01-03  627.099976  4650.660156  4.657  4.915  1759.369995  787.419983   
2007-01-04  623.900024  4673.069824  4.605  4.900  1792.910034  789.950012   
2007-01-05  604.900024  4612.350098  4.644  4.910  1785.300049  775.869995   
2007-01-08  607.500000  4624.180176  4.658  4.910  1787.140015  776.989990   
2007-01-09  613.099976  4632.660156  4.655  4.945  1795.630005  778.330017   
2007-01-10  611.599976  4641.470215  4.671  4.950  1816.150024  778.869995   
2007-01-11  612.400024  4693.040039  4.724  4.965  1834.859985  788.450012   
2007-01-12  625.500000  4760.270020  4.759  4.955  1844.810059  794.260010   
2007-01-16  624.599976  4861.839844  4.737  4.955  1842.439941  791.479980   
2007-01-17  632.099976  4813.620117  4.778  4.980  1827.680054  

# Fetching Data from FRED

##### Varible Description 

Below is a description of the variables to be fetched from FRED:

US GDP (GDP): Gross Domestic Product of the United States. GDP is the monetary value of all finished goods and services produced within a country's borders in a specific time frame. It provides an economic snapshot of the country, serving as a gauge of the country's economic health.

CPI (CPIAUCNS): Consumer Price Index for All Urban Consumers. This measures the average change over time in the prices paid by urban consumers for a market basket of consumer goods and services.

Core Inflation (CPILFESL): This is the Consumer Price Index for All Urban Consumers: All Items Less Food and Energy. It represents the CPI after removing the volatile sectors like food and energy. It provides a clearer picture of underlying, long-term trends in inflation.

Federal Funds Rate (FEDFUNDS): The interest rate at which depository institutions lend reserve balances to other depository institutions overnight. It's an important benchmark in financial markets.

Consumer Sentiment (UMCSENT): Represents the University of Michigan's Consumer Sentiment Index. This is a measure of consumer confidence and provides insights into consumer attitudes and likely consumption patterns.

Crude Oil Prices (DCOILWTICO): Represents the spot price of West Texas Intermediate (WTI) crude oil. WTI is a grade of crude oil often used as a benchmark in oil pricing.

Unemployment Rate (UNRATE): Represents the percentage of the total labor force that is unemployed but actively seeking employment and willing to work.

Housing Starts (HOUST): Number of new residential construction projects that have begun during any particular month.

Personal Saving Rate (PSAVERT): Represents the percentage of disposable income that households are saving.

In [13]:
# Load the API keys from config.json
with open('config.json', 'r') as f:
    config = json.load(f)

# Initialize the API with your key
fred_api_key = config["FRED_API_KEY"]
fred = Fred(api_key=fred_api_key)

# Define the series IDs for each dataset
series_ids = {
    # "S&P PE Ratio": "SP500PE",  # Commented out due to error
    "US GDP": "GDP",
    "CPI": "CPIAUCNS",
    "Core Inflation": "CPILFESL",
    "Federal Funds Rate": "FEDFUNDS",
    "Consumer Sentiment": "UMCSENT",
    "Crude Oil Prices": "DCOILWTICO",
    "Unemployment Rate": "UNRATE",
    "Housing Starts": "HOUST",
    "Personal Saving Rate": "PSAVERT",
    
}

# Fetch the data for each series
data_frames = {}
for name, series_id in series_ids.items():
    data_frames[name] = fred.get_series(series_id, observation_start="2007-01-01", observation_end="2023-08-10")

In [14]:
for name, df in data_frames.items():
    print(f"--- {name} ---")
    print(df.head())  # Display the first 5 rows of each dataset
    print("\n")

--- US GDP ---
2007-01-01    14215.651
2007-04-01    14402.082
2007-07-01    14564.117
2007-10-01    14715.058
2008-01-01    14706.538
dtype: float64


--- CPI ---
2007-01-01    202.416
2007-02-01    203.499
2007-03-01    205.352
2007-04-01    206.686
2007-05-01    207.949
dtype: float64


--- Core Inflation ---
2007-01-01    208.600
2007-02-01    209.135
2007-03-01    209.418
2007-04-01    209.747
2007-05-01    210.058
dtype: float64


--- Federal Funds Rate ---
2007-01-01    5.25
2007-02-01    5.26
2007-03-01    5.26
2007-04-01    5.25
2007-05-01    5.25
dtype: float64


--- Consumer Sentiment ---
2007-01-01    96.9
2007-02-01    91.3
2007-03-01    88.4
2007-04-01    87.1
2007-05-01    88.3
dtype: float64


--- Crude Oil Prices ---
2007-01-01      NaN
2007-01-02    60.77
2007-01-03    58.31
2007-01-04    55.65
2007-01-05    56.29
dtype: float64


--- Unemployment Rate ---
2007-01-01    4.6
2007-02-01    4.5
2007-03-01    4.4
2007-04-01    4.5
2007-05-01    4.4
dtype: float64


--- Ho

To consolidate the fetched data into a single DataFrame indexed by date, we use the pandas concat function. 
This will join all the individual series on their common index -> date.

In [15]:
# Use the concat function to join all series by their date index
consolidated_data = pd.concat(data_frames, axis=1)

print(consolidated_data.head())

               US GDP      CPI  Core Inflation  Federal Funds Rate  \
2007-01-01  14215.651  202.416           208.6                5.25   
2007-01-02        NaN      NaN             NaN                 NaN   
2007-01-03        NaN      NaN             NaN                 NaN   
2007-01-04        NaN      NaN             NaN                 NaN   
2007-01-05        NaN      NaN             NaN                 NaN   

            Consumer Sentiment  Crude Oil Prices  Unemployment Rate  \
2007-01-01                96.9               NaN                4.6   
2007-01-02                 NaN             60.77                NaN   
2007-01-03                 NaN             58.31                NaN   
2007-01-04                 NaN             55.65                NaN   
2007-01-05                 NaN             56.29                NaN   

            Housing Starts  Personal Saving Rate  
2007-01-01          1409.0                   2.4  
2007-01-02             NaN                   NaN  

To fill missing values in a time series dataset, the common method used is forward filling (ffill). This method fills missing values with the last available (non-missing) value. This approach works well especially for datasets like GDP which are reported annually or quarterly, as you can fill each day's missing data with the value from the last reported period until a new value becomes available.

In [16]:
consolidated_data_filled = consolidated_data.ffill()

In [17]:
print(consolidated_data_filled.head(13))

               US GDP      CPI  Core Inflation  Federal Funds Rate  \
2007-01-01  14215.651  202.416           208.6                5.25   
2007-01-02  14215.651  202.416           208.6                5.25   
2007-01-03  14215.651  202.416           208.6                5.25   
2007-01-04  14215.651  202.416           208.6                5.25   
2007-01-05  14215.651  202.416           208.6                5.25   
2007-01-08  14215.651  202.416           208.6                5.25   
2007-01-09  14215.651  202.416           208.6                5.25   
2007-01-10  14215.651  202.416           208.6                5.25   
2007-01-11  14215.651  202.416           208.6                5.25   
2007-01-12  14215.651  202.416           208.6                5.25   
2007-01-15  14215.651  202.416           208.6                5.25   
2007-01-16  14215.651  202.416           208.6                5.25   
2007-01-17  14215.651  202.416           208.6                5.25   

            Consume

# Merging into one DataFrame

To merge the two dataframes by date, we'll use the merge function provided by pandas. Since the 'Date' column serves as the index for both dataframes, the merge will be based on this index.

The how='left' argument ensures that ensures that the merged dataframe retains only the rows with indices (dates) from merged_data.

In [18]:
# Merging the dataframes on Date and keeping only dates present in merged_data
merged_dataframe = merged_data.merge(consolidated_data_filled, left_index=True, right_index=True, how='left')


In [19]:
print(merged_dataframe.head(13))

                  GC=F         ^DJT   ^FVX   ^IRX         ^NDX        ^RUT  \
Date                                                                         
2007-01-02  635.200012          NaN    NaN    NaN          NaN         NaN   
2007-01-03  627.099976  4650.660156  4.657  4.915  1759.369995  787.419983   
2007-01-04  623.900024  4673.069824  4.605  4.900  1792.910034  789.950012   
2007-01-05  604.900024  4612.350098  4.644  4.910  1785.300049  775.869995   
2007-01-08  607.500000  4624.180176  4.658  4.910  1787.140015  776.989990   
2007-01-09  613.099976  4632.660156  4.655  4.945  1795.630005  778.330017   
2007-01-10  611.599976  4641.470215  4.671  4.950  1816.150024  778.869995   
2007-01-11  612.400024  4693.040039  4.724  4.965  1834.859985  788.450012   
2007-01-12  625.500000  4760.270020  4.759  4.955  1844.810059  794.260010   
2007-01-16  624.599976  4861.839844  4.737  4.955  1842.439941  791.479980   
2007-01-17  632.099976  4813.620117  4.778  4.980  1827.680054  

In [20]:
# Save merged_dataframe to a .csv file
merged_dataframe.to_csv('merged_dataframe.csv')


## Missing Values

In [21]:
# Checking for missing values in the dataframe
missing_values = merged_dataframe.isnull().sum()

# Filtering out columns that have missing values
columns_with_missing_values = missing_values[missing_values > 0]

# Calculating the percentage of missing values for each column
percentage_missing = (columns_with_missing_values / len(merged_dataframe)) * 100

# Displaying the results
print("Columns with missing values:")
print(columns_with_missing_values)
print("\nPercentage of missing values for each column:")
print(percentage_missing)


Columns with missing values:
GC=F           3
^DJT           1
^FVX           4
^IRX           4
^NDX           1
^RUT           1
^TNX           4
^VIX           1
^W5000         8
Volume_^DJT    1
Volume_^NDX    1
Volume_^RUT    1
dtype: int64

Percentage of missing values for each column:
GC=F           0.071770
^DJT           0.023923
^FVX           0.095694
^IRX           0.095694
^NDX           0.023923
^RUT           0.023923
^TNX           0.095694
^VIX           0.023923
^W5000         0.191388
Volume_^DJT    0.023923
Volume_^NDX    0.023923
Volume_^RUT    0.023923
dtype: float64


In [22]:
# Locating rows with missing values
rows_with_missing_values = merged_dataframe[merged_dataframe.isnull().any(axis=1)]

print(rows_with_missing_values)


                   GC=F          ^DJT   ^FVX   ^IRX          ^NDX  \
Date                                                                
2007-01-02   635.200012           NaN    NaN    NaN           NaN   
2010-10-11  1353.300049   4630.740234    NaN    NaN   2026.979980   
2016-04-01  1222.199951   7887.770020  1.242  0.218   4532.080078   
2016-05-03  1290.699951   7828.319824  1.257  0.203   4341.390137   
2016-05-31  1214.800049   7798.350098  1.359  0.283   4523.890137   
2016-07-22  1323.099976   7965.979980  1.120  0.303   4666.069824   
2016-08-23  1340.599976   7923.399902  1.143  0.305   4818.479980   
2016-08-25  1320.099976   7868.029785  1.157  0.305   4775.430176   
2016-10-10          NaN   8137.089844    NaN    NaN   4893.770020   
2016-11-11          NaN   8578.650391    NaN    NaN   4751.950195   
2018-01-29          NaN  11114.230469  2.495  1.375   6988.319824   
2023-06-01  1978.000000  13880.169922  3.701  5.203  14441.509766   

                   ^RUT   ^TNX   

In [23]:
print(merged_dataframe.shape)

(4180, 22)


For the 2007-01-02 row:
This seems to be a typical case where the data for the first day is often not available for many metrics, since the prior data point (possibly from the end of 2006) is not included in this dataset. This is a common issue when dealing with financial and economic datasets, especially when pulling from different sources.

For the 2010-10-11 row:
It's possible that 2010-10-11 was a holiday for bond markets (e.g., Columbus Day), which can explain why some bond-related indicators (^FVX, ^IRX, ^TNX) are missing. Equity markets, however, often remain open on Columbus Day, which explains why there are values for some columns.